# Example usage

To use `centopy` in a project:

In [1]:
import centopy

print(centopy.__version__)

0.2.2


## Text file operation

The `centopy.FilesManager` class works as an adapter to the use of context manager when making CRUD operations, like:

```python
with open('file.txt') as file_:
    file.write('hello')
```

So, every action on files closes it after perfoming the intended operation.

In [2]:
wdir = 'data'
manager = centopy.FilesManager(folder_path=wdir)

# now manager.folder_path is a pathlib.Path object
print(type(manager.folder_path))

<class 'pathlib.PosixPath'>


In [3]:
# Write text to a file
file_contents = "Hello, Centopy!"
manager.write("hello.txt", file_contents)

# Read the written file
read_contents = manager.read("hello.txt")
print(read_contents)  # Output: Hello, Centopy!


Hello, Centopy!


## Binary file operation

In [4]:
# Initial binary data
initial_data = b"\x48\x65\x6C\x6C\x6F"  # Corresponds to "Hello"
manager.writeb("data.bin", initial_data)

# Append binary data
new_data = b"\x20\x77\x6F\x72\x6C\x64"  # Corresponds to " world"
manager.appendb("data.bin", new_data)

# Read the appended binary data
appended_data = manager.readb("data.bin")
print(appended_data)  # Output: b'Hello world'


b'Hello world'


## List files

In [5]:
manager.list_files()

['file.customzip', 'data.bin', 'binary_data.bin', 'hello.txt']

## Files state

In [6]:
file_name = 'hello2.txt'

file_contents = "Hello again, Centopy!"
manager.write(file_name, file_contents)
print(manager.get_file_state(file_name))

read_contents = manager.read(file_name)
print(manager.get_file_state(file_name))

manager.delete_file(file_name)
print(manager.get_file_state(file_name))

saved
loaded
deleted



A `loaded` state simply means that the contents of the file were saved
in a variable

### Appending data

In [7]:
file_name = 'hello.txt'
content = "\nLet's make some changes!"
manager.append(file_name=file_name, file_contents=content)

print(f">>{file_name}'s state: {manager.get_file_state(file_name)}")

entire_content = manager.read(file_name)

print(f">>{file_name}'s updated content: \n{entire_content}")
print(f">>{file_name}'s new state: {manager.get_file_state(file_name)}")

>>hello.txt's state: saved
>>hello.txt's updated content: 
Hello, Centopy!
Let's make some changes!
>>hello.txt's new state: loaded


### String representation

In [8]:
print(manager)

Working dir: /home/bessa/Documents/projetos/centopy/docs/data
file.customzip: unknown
data.bin: loaded
binary_data.bin: unknown
hello.txt: loaded


## Compress (zip) file operations

### Writing, appending and removing files

In [9]:
import shutil
from pathlib import Path

In [10]:
wdir = 'data'
shutil.rmtree(wdir)

file_name = 'data.txt'

# Create a Compressor object
archive = centopy.Compressor('file', extension='customzip', wdir='data')

# Write text content to a new file and add it to the archive
# overwrite if it's already a member of archive
archive.write(file_name, "Hello, this is a new file!")

# Append content to an existing file in the archive
archive.append(file_name, "\nAdditional data appended.")

# Read the content of a file from the archive
file_content = archive.read("data.txt")
print(f">> Appended content:\n{file_content}")

# archive has it's own FilesManager object
print(f">> Files in folder 'data': {archive.manager.list_files()}")

>> Appended content:
Hello, this is a new file!
Additional data appended.
>> Files in folder 'data': ['file.customzip', 'data.txt']


### Binary Data Compression

In [11]:
# Write binary content to a new file and add it to the archive
binary_data = b'\x00\x01\x02\x03\x04'
archive.writeb("binary_data.bin", binary_data)

# Append binary content to an existing file in the archive
new_binary_data = b'\x05\x06\x07'
archive.appendb("binary_data.bin", new_binary_data)

# Read the binary content of a file from the archive
binary_content = archive.read("binary_data.bin", as_text=False)

print("Binary content:", binary_content)

Binary content: b'\x00\x01\x02\x03\x04\x05\x06\x07'


### List members of the archive

In [12]:
print(archive.namelist())

['binary_data.bin', 'data.txt']


### Removing member

In [13]:
archive.remove('binary_data.bin')
print(archive.namelist())

['data.txt']


### Updating member

In [14]:
member = 'data.txt'
content = 'Updated content'

with open(archive.manager.folder_path / member, mode='w', encoding='utf-8') as updated:
    updated.write(content)

archive.update(member, delete_source=True)

file_content = archive.read("data.txt")
print(f">> Updated content:\n{file_content}")
print(f">> See that {member} was deleted from wdir: {archive.manager.list_files()}")

>> Updated content:
Updated content
>> See that data.txt was deleted from wdir: ['file.customzip', 'binary_data.bin']
